In [38]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
file_path = "financial_transactions.csv"
df = pd.read_csv(file_path)
users_df = df[['customer_id']].drop_duplicates().reset_index(drop=True)
users_df['user_id'] = users_df.index + 1
users_df['name'] = "User_" + users_df['customer_id'].astype(str)
users_df['email'] = users_df['name'].str.lower() + "@example.com"
users_df['phone_number'] = '123-456-' + users_df['customer_id'].astype(str).str[-4:]
users_df = users_df[['user_id', 'customer_id', 'name', 'email', 'phone_number']]
categories_data = ['Streaming', 'Health', 'Software', 'Utilities', 'Other']
categories_df = pd.DataFrame({'category_id': range(1, len(categories_data) + 1), 'category_name': categories_data})
services_data = [
    (1, "Netflix", 1), (2, "Amazon Prime", 1), (3, "Spotify", 1), 
    (4, "Gym Membership", 2), (5, "Adobe Photoshop", 3), 
    (6, "Electricity Bill", 4), (7, "Disney+", 1), (8, "Apple Music", 1), 
    (9, "YouTube Premium", 1)
]
services_df = pd.DataFrame(services_data, columns=['service_id', 'service_name', 'category_id'])
services_df = services_df[services_df['category_id'].isin(categories_df['category_id'])]
billing_cycles_df = pd.DataFrame({'billing_cycle_id': [1, 2], 'cycle_name': ['Monthly', 'Yearly']})
subscription_status_df = pd.DataFrame({'status_id': [1, 2, 3], 'status_name': ['Active', 'Paused', 'Canceled']})
payment_methods_df = pd.DataFrame({'payment_method_id': [1, 2, 3, 4], 
                                   'method_name': ['Credit Card', 'PayPal', 'Bank Transfer', 'Crypto']})
subscriptions_df = df[['customer_id', 'amount', 'date']].drop_duplicates().reset_index(drop=True)
subscriptions_df['subscription_id'] = subscriptions_df.index + 1
valid_users = users_df.set_index('customer_id')['user_id'].to_dict()
subscriptions_df['user_id'] = subscriptions_df['customer_id'].map(valid_users)
subscriptions_df = subscriptions_df.dropna(subset=['user_id'])  # Remove invalid users
subscriptions_df['user_id'] = subscriptions_df['user_id'].astype(int)
valid_service_ids = services_df['service_id'].tolist()
subscriptions_df['service_id'] = np.random.choice(valid_service_ids, size=len(subscriptions_df))
valid_billing_cycles = billing_cycles_df['billing_cycle_id'].tolist()
subscriptions_df['billing_cycle_id'] = np.random.choice(valid_billing_cycles, size=len(subscriptions_df))
valid_status_ids = subscription_status_df['status_id'].tolist()
subscriptions_df['status_id'] = np.random.choice(valid_status_ids, size=len(subscriptions_df))
subscriptions_df['next_payment_date'] = pd.to_datetime(subscriptions_df['date']) + timedelta(days=30)
subscriptions_df['auto_renew'] = np.random.choice([True, False], size=len(subscriptions_df))
subscriptions_df = subscriptions_df.drop(columns=['customer_id'])
payments_df = subscriptions_df.sample(frac=0.7, random_state=42)  
payments_df['payment_id'] = payments_df.index + 1
payments_df['payment_date'] = payments_df['next_payment_date'] - timedelta(days=30)
valid_payment_methods = payment_methods_df['payment_method_id'].tolist()
payments_df['payment_method_id'] = np.random.choice(valid_payment_methods, size=len(payments_df))
payments_df['status'] = np.random.choice(['Completed', 'Failed', 'Pending'], size=len(payments_df))
payments_df = payments_df[payments_df['subscription_id'].isin(subscriptions_df['subscription_id'])]
notifications_df = subscriptions_df.sample(frac=0.5, random_state=42)  
notifications_df['notification_id'] = notifications_df.index + 1
notifications_df['message'] = "Your payment for " + notifications_df['billing_cycle_id'].map(lambda x: 'Monthly' if x == 1 else 'Yearly') + " plan is due soon."
notifications_df['status'] = np.random.choice(['Sent', 'Pending'], size=len(notifications_df))
notifications_df['sent_at'] = datetime.now()
notifications_df = notifications_df[notifications_df['subscription_id'].isin(subscriptions_df['subscription_id'])]
users_df.to_csv("transformed_users.csv", index=False)
categories_df.to_csv("transformed_categories.csv", index=False)
services_df.to_csv("transformed_services.csv", index=False)
billing_cycles_df.to_csv("transformed_billing_cycles.csv", index=False)
subscription_status_df.to_csv("transformed_subscription_status.csv", index=False)
subscriptions_df.to_csv("transformed_subscriptions.csv", index=False)
payment_methods_df.to_csv("transformed_payment_methods.csv", index=False)
payments_df.to_csv("transformed_payments.csv", index=False)
notifications_df.to_csv("transformed_notifications.csv", index=False)
print(" Transformed datasets saved as CSV files and all foreign key constraints are satisfied!")

 Transformed datasets saved as CSV files and all foreign key constraints are satisfied!


In [32]:
categories_df

,category_id,category_name
0,1,Streaming
1,2,Health
2,3,Software
3,4,Utilities
4,5,Other


In [40]:
import pandas as pd
df = pd.read_csv("transformed_notifications.csv") 
subscription_metadata = df[[
    "subscription_id", "user_id", "service_id", "billing_cycle_id",
    "status_id", "next_payment_date", "auto_renew"
]].drop_duplicates()
notification_messages = df[[
    "notification_id", "subscription_id", "amount", "date", "message", "status", "sent_at"
]]
subscription_metadata.to_csv("Subscription_Metadata_CLEANED.csv", index=False)
notification_messages.to_csv("Notification_Messages_CLEANED.csv", index=False)

print("Columns reassigned and saved correctly.")

Columns reassigned and saved correctly.


In [22]:
subscription_metadata

,subscription_id,user_id,service_id,billing_cycle_id,status_id,next_payment_date,auto_renew
0,75722,618,1,1,1,2019-07-07,False
1,80185,378,3,1,3,2021-02-15,False
2,19865,604,8,2,3,2022-10-16,True
3,76700,785,8,2,2,2018-06-18,True
4,92992,716,4,1,1,2019-07-09,True
...,...,...,...,...,...,...,...
49995,33073,432,6,2,2,2020-02-23,False
49996,95600,603,6,2,3,2022-01-14,False
49997,28007,198,7,1,2,2020-07-07,True
49998,90488,103,2,2,1,2020-08-14,True


In [24]:
notification_messages

,notification_id,subscription_id,amount,date,message,status,sent_at
0,75722,75722,9397.31,2019-06-07,Your payment for Monthly plan is due soon.,Sent,2025-04-19 01:26:01.861657
1,80185,80185,2675.14,2021-01-16,Your payment for Monthly plan is due soon.,Pending,2025-04-19 01:26:01.861657
2,19865,19865,7784.79,2022-09-16,Your payment for Yearly plan is due soon.,Sent,2025-04-19 01:26:01.861657
3,76700,76700,6710.96,2018-05-19,Your payment for Yearly plan is due soon.,Sent,2025-04-19 01:26:01.861657
4,92992,92992,2296.84,2019-06-09,Your payment for Monthly plan is due soon.,Sent,2025-04-19 01:26:01.861657
...,...,...,...,...,...,...,...
49995,33073,33073,9297.62,2020-01-24,Your payment for Yearly plan is due soon.,Sent,2025-04-19 01:26:01.861657
49996,95600,95600,3455.63,2021-12-15,Your payment for Yearly plan is due soon.,Pending,2025-04-19 01:26:01.861657
49997,28007,28007,474.57,2020-06-07,Your payment for Monthly plan is due soon.,Pending,2025-04-19 01:26:01.861657
49998,90488,90488,7654.53,2020-07-15,Your payment for Yearly plan is due soon.,Pending,2025-04-19 01:26:01.861657


In [42]:
import pandas as pd
df = pd.read_csv("transformed_payments.csv") 
payments_info = df[[
    "payment_id", "subscription_id", "amount", "date", "payment_date",
    "payment_method_id", "status"
]]
subscription_info = df[[
    "subscription_id", "user_id", "service_id", "billing_cycle_id",
    "status_id", "next_payment_date", "auto_renew"
]].drop_duplicates()
payments_info.to_csv("Payments_Info_CLEANED.csv", index=False)
subscription_info.to_csv("Subscription_Info_CLEANED.csv", index=False)
print(" Restructuring complete. Ready to COPY into PostgreSQL.")

 Restructuring complete. Ready to COPY into PostgreSQL.


In [12]:
payments_info

,payment_id,subscription_id,amount,date,payment_date,payment_method_id,status
0,75722,75722,9397.31,2019-06-07,2019-06-07,3,Pending
1,80185,80185,2675.14,2021-01-16,2021-01-16,1,Failed
2,19865,19865,7784.79,2022-09-16,2022-09-16,2,Pending
3,76700,76700,6710.96,2018-05-19,2018-05-19,4,Completed
4,92992,92992,2296.84,2019-06-09,2019-06-09,3,Pending
...,...,...,...,...,...,...,...
69995,7076,7076,2648.06,2020-09-14,2020-09-14,3,Pending
69996,51073,51073,8143.18,2021-04-06,2021-04-06,1,Pending
69997,71583,71583,5294.08,2019-02-04,2019-02-04,4,Failed
69998,21636,21636,6391.87,2019-01-26,2019-01-26,3,Pending


In [10]:
subscription_info

,subscription_id,user_id,service_id,billing_cycle_id,status_id,next_payment_date,auto_renew
0,75722,618,1,1,1,2019-07-07,False
1,80185,378,3,1,3,2021-02-15,False
2,19865,604,8,2,3,2022-10-16,True
3,76700,785,8,2,2,2018-06-18,True
4,92992,716,4,1,1,2019-07-09,True
...,...,...,...,...,...,...,...
69995,7076,587,4,1,3,2020-10-14,False
69996,51073,260,5,1,2,2021-05-06,False
69997,71583,790,3,1,3,2019-03-06,False
69998,21636,103,9,1,3,2019-02-25,False


In [16]:
import pandas as pd
df = pd.read_csv("Payments_Info_CLEANED.csv") 
equal_rows = df["date"] == df["payment_date"]
all_equal = equal_rows.all()
print("Are all 'date' and 'payment_date' values the same?:", all_equal)

Are all 'date' and 'payment_date' values the same?: True


In [50]:
import pandas as pd
df = pd.read_csv("transformed_users.csv") 
null_user_id = df[df['user_id'].isnull()]
print("\n NULLs found in PRIMARY KEY 'user_id':\n", null_user_id)
duplicate_user_id = df[df.duplicated('user_id', keep=False)]
print("\n Duplicate 'user_id' values found (violates PRIMARY KEY):\n", duplicate_user_id)
not_null_violations = df[df[['customer_id', 'name', 'email', 'phone_number']].isnull().any(axis=1)]
print("\n Records with NULLs in NOT NULL fields:\n", not_null_violations)
duplicate_emails = df[df.duplicated('email', keep=False)]
print("\n Duplicate emails found:\n", duplicate_emails)
duplicate_contacts = df[df.duplicated('phone_number', keep=False)]
print("\n Duplicate phone numbers found:\n", duplicate_contacts)
invalid_email_format = df[~df['email'].str.contains(r'^[^@]+@[^@]+\.[^@]+$', regex=True, na=False)]
print("\n Invalid email formats:\n", invalid_email_format)
print("\n Validation complete.")


 NULLs found in PRIMARY KEY 'user_id':
 Empty DataFrame
Columns: [user_id, customer_id, name, email, phone_number]
Index: []

 Duplicate 'user_id' values found (violates PRIMARY KEY):
 Empty DataFrame
Columns: [user_id, customer_id, name, email, phone_number]
Index: []

 Records with NULLs in NOT NULL fields:
 Empty DataFrame
Columns: [user_id, customer_id, name, email, phone_number]
Index: []

 Duplicate emails found:
 Empty DataFrame
Columns: [user_id, customer_id, name, email, phone_number]
Index: []

 Duplicate phone numbers found:
 Empty DataFrame
Columns: [user_id, customer_id, name, email, phone_number]
Index: []

 Invalid email formats:
 Empty DataFrame
Columns: [user_id, customer_id, name, email, phone_number]
Index: []

 Validation complete.


In [46]:
import pandas as pd
df = pd.read_csv("transformed_categories.csv")  
null_pk = df[df['category_id'].isnull()]
print("\n NULL values found in PRIMARY KEY 'category_id':\n", null_pk)
duplicate_pk = df[df.duplicated('category_id', keep=False)]
print("\n Duplicate category_id values found (violates PRIMARY KEY uniqueness):\n", duplicate_pk)
not_null_violations = df[df['category_name'].isnull()]
print("\n NULLs in NOT NULL field 'category_name':\n", not_null_violations)
duplicate_categories = df[df.duplicated('category_name', keep=False)]
print("\n Duplicate category_name values found:\n", duplicate_categories)
print("\n Validation complete.")


 NULL values found in PRIMARY KEY 'category_id':
 Empty DataFrame
Columns: [category_id, category_name]
Index: []

 Duplicate category_id values found (violates PRIMARY KEY uniqueness):
 Empty DataFrame
Columns: [category_id, category_name]
Index: []

 NULLs in NOT NULL field 'category_name':
 Empty DataFrame
Columns: [category_id, category_name]
Index: []

 Duplicate category_name values found:
 Empty DataFrame
Columns: [category_id, category_name]
Index: []

 Validation complete.


In [52]:
import pandas as pd
df = pd.read_csv("transformed_billing_cycles.csv")
null_ids = df[df['billing_cycle_id'].isnull()]
print("\nNULLs in billing_cycle_id (violates PRIMARY KEY):\n", null_ids)
duplicate_ids = df[df.duplicated('billing_cycle_id', keep=False)]
print("\nDuplicate billing_cycle_id (violates PRIMARY KEY):\n", duplicate_ids)
null_names = df[df['cycle_name'].isnull()]
print("\nNULLs in cycle_name (violates NOT NULL):\n", null_names)
duplicate_names = df[df.duplicated('cycle_name', keep=False)]
print("\nDuplicate cycle_name values (violates UNIQUE):\n", duplicate_names)
invalid_values = df[~df['cycle_name'].isin(['Monthly', 'Yearly'])]
print("\nInvalid cycle_name values (violates CHECK constraint):\n", invalid_values)
print("\nBilling_Cycles validation complete.")


NULLs in billing_cycle_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [billing_cycle_id, cycle_name]
Index: []

Duplicate billing_cycle_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [billing_cycle_id, cycle_name]
Index: []

NULLs in cycle_name (violates NOT NULL):
 Empty DataFrame
Columns: [billing_cycle_id, cycle_name]
Index: []

Duplicate cycle_name values (violates UNIQUE):
 Empty DataFrame
Columns: [billing_cycle_id, cycle_name]
Index: []

Invalid cycle_name values (violates CHECK constraint):
 Empty DataFrame
Columns: [billing_cycle_id, cycle_name]
Index: []

Billing_Cycles validation complete.


In [54]:
import pandas as pd
df = pd.read_csv("transformed_subscription_status.csv")
null_ids = df[df['status_id'].isnull()]
print("\nNULLs in status_id (violates PRIMARY KEY):\n", null_ids)
duplicate_ids = df[df.duplicated('status_id', keep=False)]
print("\nDuplicate status_id (violates PRIMARY KEY):\n", duplicate_ids)
null_names = df[df['status_name'].isnull()]
print("\nNULLs in status_name (violates NOT NULL constraint):\n", null_names)
duplicate_names = df[df.duplicated('status_name', keep=False)]
print("\nDuplicate status_name (violates UNIQUE constraint):\n", duplicate_names)
valid_values = ['Active', 'Paused', 'Canceled']
invalid_status_names = df[~df['status_name'].isin(valid_values)]
print("\nInvalid status_name values (violates CHECK constraint):\n", invalid_status_names)
print("\nSubscription_Status table validation complete.")


NULLs in status_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [status_id, status_name]
Index: []

Duplicate status_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [status_id, status_name]
Index: []

NULLs in status_name (violates NOT NULL constraint):
 Empty DataFrame
Columns: [status_id, status_name]
Index: []

Duplicate status_name (violates UNIQUE constraint):
 Empty DataFrame
Columns: [status_id, status_name]
Index: []

Invalid status_name values (violates CHECK constraint):
 Empty DataFrame
Columns: [status_id, status_name]
Index: []

Subscription_Status table validation complete.


In [56]:
import pandas as pd
df = pd.read_csv("transformed_payment_methods.csv")
null_ids = df[df['payment_method_id'].isnull()]
print("\nNULLs in payment_method_id (violates PRIMARY KEY):\n", null_ids)
duplicate_ids = df[df.duplicated('payment_method_id', keep=False)]
print("\nDuplicate payment_method_id (violates PRIMARY KEY):\n", duplicate_ids)
null_names = df[df['method_name'].isnull()]
print("\nNULLs in method_name (violates NOT NULL constraint):\n", null_names)
duplicate_names = df[df.duplicated('method_name', keep=False)]
print("\nDuplicate method_name (violates UNIQUE constraint):\n", duplicate_names)
print("\nPayment_Methods table validation complete.")


NULLs in payment_method_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [payment_method_id, method_name]
Index: []

Duplicate payment_method_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [payment_method_id, method_name]
Index: []

NULLs in method_name (violates NOT NULL constraint):
 Empty DataFrame
Columns: [payment_method_id, method_name]
Index: []

Duplicate method_name (violates UNIQUE constraint):
 Empty DataFrame
Columns: [payment_method_id, method_name]
Index: []

Payment_Methods table validation complete.


In [58]:
import pandas as pd
services_df = pd.read_csv("transformed_services.csv")
categories_df = pd.read_csv("transformed_categories.csv")
null_ids = services_df[services_df['service_id'].isnull()]
print("\n NULLs in service_id (violates PRIMARY KEY):\n", null_ids)
duplicate_ids = services_df[services_df.duplicated('service_id', keep=False)]
print("\n Duplicate service_id (violates PRIMARY KEY):\n", duplicate_ids)
null_names = services_df[services_df['service_name'].isnull()]
print("\n NULLs in service_name (violates NOT NULL):\n", null_names)
duplicate_names = services_df[services_df.duplicated('service_name', keep=False)]
print("\n Duplicate service_name (violates UNIQUE):\n", duplicate_names)
null_categories = services_df[services_df['category_id'].isnull()]
print("\n NULLs in category_id (violates NOT NULL):\n", null_categories)
invalid_fk = services_df[~services_df['category_id'].isin(categories_df['category_id'])]
print("\n category_id values not found in Categories table (violates FOREIGN KEY):\n", invalid_fk)
print("\n Services table validation complete.")


 NULLs in service_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [service_id, service_name, category_id]
Index: []

 Duplicate service_id (violates PRIMARY KEY):
 Empty DataFrame
Columns: [service_id, service_name, category_id]
Index: []

 NULLs in service_name (violates NOT NULL):
 Empty DataFrame
Columns: [service_id, service_name, category_id]
Index: []

 Duplicate service_name (violates UNIQUE):
 Empty DataFrame
Columns: [service_id, service_name, category_id]
Index: []

 NULLs in category_id (violates NOT NULL):
 Empty DataFrame
Columns: [service_id, service_name, category_id]
Index: []

 category_id values not found in Categories table (violates FOREIGN KEY):
 Empty DataFrame
Columns: [service_id, service_name, category_id]
Index: []

 Services table validation complete.


In [60]:
import pandas as pd
subscriptions = pd.read_csv("transformed_subscriptions.csv")
users = pd.read_csv("transformed_users.csv")
services = pd.read_csv("transformed_services.csv")
billing_cycles = pd.read_csv("transformed_billing_cycles.csv")
statuses = pd.read_csv("transformed_subscription_status.csv")
print("\n Null subscription_id:\n", subscriptions[subscriptions['subscription_id'].isnull()])
print("\n Duplicate subscription_id:\n", subscriptions[subscriptions.duplicated('subscription_id', keep=False)])
not_null_fields = ['user_id', 'service_id', 'billing_cycle_id', 'status_id', 'amount', 'date', 'next_payment_date', 'auto_renew']
nulls = subscriptions[subscriptions[not_null_fields].isnull().any(axis=1)]
print("\n Rows with NULLs in NOT NULL fields:\n", nulls)
invalid_amounts = subscriptions[subscriptions['amount'] < 0]
print("\n Invalid amount values (must be >= 0):\n", invalid_amounts)
invalid_dates = subscriptions[pd.to_datetime(subscriptions['next_payment_date']) < pd.to_datetime(subscriptions['date'])]
print("\n next_payment_date is before subscription date:\n", invalid_dates)
invalid_user_ids = subscriptions[~subscriptions['user_id'].isin(users['user_id'])]
print("\n Invalid user_id (not found in Users):\n", invalid_user_ids)
invalid_service_ids = subscriptions[~subscriptions['service_id'].isin(services['service_id'])]
print("\n Invalid service_id (not found in Services):\n", invalid_service_ids)
invalid_cycle_ids = subscriptions[~subscriptions['billing_cycle_id'].isin(billing_cycles['billing_cycle_id'])]
print("\n Invalid billing_cycle_id (not found in Billing_Cycles):\n", invalid_cycle_ids)
invalid_status_ids = subscriptions[~subscriptions['status_id'].isin(statuses['status_id'])]
print("\n Invalid status_id (not found in Subscription_Status):\n", invalid_status_ids)
print("\n Subscription table validation complete.")


 Null subscription_id:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew]
Index: []

 Duplicate subscription_id:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew]
Index: []

 Rows with NULLs in NOT NULL fields:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew]
Index: []

 Invalid amount values (must be >= 0):
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew]
Index: []

 next_payment_date is before subscription date:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew]
Index: []

 Invalid user_id (not found in Users):
 Empty DataFrame
Columns: [amount, 

In [62]:
import pandas as pd
payments = pd.read_csv("transformed_payments.csv")
subscriptions = pd.read_csv("transformed_subscriptions.csv")
users = pd.read_csv("transformed_users.csv")
services = pd.read_csv("transformed_services.csv")
billing_cycles = pd.read_csv("transformed_billing_cycles.csv")
statuses = pd.read_csv("transformed_subscription_status.csv")
methods = pd.read_csv("transformed_payment_methods.csv")
print("\n Null payment_id:\n", payments[payments['payment_id'].isnull()])
print("\n Duplicate payment_id:\n", payments[payments.duplicated('payment_id', keep=False)])
print("\n Invalid amount (amount < 0):\n", payments[payments['amount'] < 0])
payments['date'] = pd.to_datetime(payments['date'], errors='coerce')
payments['next_payment_date'] = pd.to_datetime(payments['next_payment_date'], errors='coerce')
print("\n next_payment_date < date:\n", payments[payments['next_payment_date'] < payments['date']])
valid_statuses = ['Completed', 'Failed', 'Pending']
print("\n Invalid status:\n", payments[~payments['status'].isin(valid_statuses)])
required_fields = ['amount', 'date', 'subscription_id', 'user_id', 'service_id',
                   'billing_cycle_id', 'status_id', 'next_payment_date', 'auto_renew',
                   'payment_date', 'payment_method_id', 'status']
print("\n Rows with NULLs in required fields:\n", payments[payments[required_fields].isnull().any(axis=1)])
print("\n Invalid subscription_id:\n", payments[~payments['subscription_id'].isin(subscriptions['subscription_id'])])
print("\n Invalid user_id:\n", payments[~payments['user_id'].isin(users['user_id'])])
print("\n Invalid service_id:\n", payments[~payments['service_id'].isin(services['service_id'])])
print("\n Invalid billing_cycle_id:\n", payments[~payments['billing_cycle_id'].isin(billing_cycles['billing_cycle_id'])])
print("\n Invalid status_id:\n", payments[~payments['status_id'].isin(statuses['status_id'])])
print("\n Invalid payment_method_id:\n", payments[~payments['payment_method_id'].isin(methods['payment_method_id'])])
print("\n Payments table validation complete.")


 Null payment_id:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, payment_id, payment_date, payment_method_id, status]
Index: []

 Duplicate payment_id:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, payment_id, payment_date, payment_method_id, status]
Index: []

 Invalid amount (amount < 0):
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, payment_id, payment_date, payment_method_id, status]
Index: []

 next_payment_date < date:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, payment_id, payment_date, payment_method_id, status]
Index: []

 Invalid status:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, servi

In [64]:
import pandas as pd
notifications = pd.read_csv("transformed_notifications.csv")
subscriptions = pd.read_csv("transformed_subscriptions.csv")
users = pd.read_csv("transformed_users.csv")
services = pd.read_csv("transformed_services.csv")
billing = pd.read_csv("transformed_billing_cycles.csv")
statuses = pd.read_csv("transformed_subscription_status.csv")
print("\n Null notification_id:\n", notifications[notifications['notification_id'].isnull()])
print("\n Duplicate notification_id:\n", notifications[notifications.duplicated('notification_id', keep=False)])
print("\n Null 'message' or 'status':\n", notifications[notifications[['message', 'status']].isnull().any(axis=1)])
print("\n amount < 0:\n", notifications[notifications['amount'] < 0])
valid_statuses = ['Sent', 'Pending']
print("\n Invalid status values:\n", notifications[~notifications['status'].isin(valid_statuses)])
print("\n Invalid subscription_id:\n", notifications[~notifications['subscription_id'].isin(subscriptions['subscription_id'])])
print("\n Invalid user_id:\n", notifications[~notifications['user_id'].isin(users['user_id'])])
print("\n Invalid service_id:\n", notifications[~notifications['service_id'].isin(services['service_id'])])
print("\n Invalid billing_cycle_id:\n", notifications[~notifications['billing_cycle_id'].isin(billing['billing_cycle_id'])])
print("\n Invalid status_id:\n", notifications[~notifications['status_id'].isin(statuses['status_id'])])
print("\n Notification table validation complete.")


 Null notification_id:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, notification_id, message, status, sent_at]
Index: []

 Duplicate notification_id:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, notification_id, message, status, sent_at]
Index: []

 Null 'message' or 'status':
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, notification_id, message, status, sent_at]
Index: []

 amount < 0:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, next_payment_date, auto_renew, notification_id, message, status, sent_at]
Index: []

 Invalid status values:
 Empty DataFrame
Columns: [amount, date, subscription_id, user_id, service_id, billing_cycle_id, status_id, ne

In [33]:
import pandas as pd
subscription_info = pd.read_csv("Subscription_Info_CLEANED.csv")
payments_info = pd.read_csv("Payments_Info_CLEANED.csv")
subscription_metadata = pd.read_csv("Subscription_Metadata_CLEANED.csv")
notification_messages = pd.read_csv("Notification_Messages_CLEANED.csv")
pk_violations = {}
def check_pk(df, key, name):
    duplicates = df[df.duplicated(subset=[key])]
    if not duplicates.empty:
        pk_violations[name] = duplicates
        print(f" PK violation in {name} (duplicate {key}):\n", duplicates, "\n")
    else:
        print(f" No primary key violation in {name}.\n")

check_pk(subscription_info, "subscription_id", "Subscription_Info")
check_pk(payments_info, "payment_id", "Payments_Info")
check_pk(subscription_metadata, "subscription_id", "Subscription_Metadata")
check_pk(notification_messages, "notification_id", "Notification_Messages")
invalid_payments_fk = payments_info[
    ~payments_info["subscription_id"].isin(subscription_info["subscription_id"])
]
invalid_notifications_fk = notification_messages[
    ~notification_messages["subscription_id"].isin(subscription_metadata["subscription_id"])
]

print(" FK Check: Payments_Info → Subscription_Info")
if invalid_payments_fk.empty:
    print(" All foreign keys in Payments_Info are valid.\n")
else:
    print(" Invalid foreign keys in Payments_Info:\n", invalid_payments_fk)

print(" FK Check: Notification_Messages → Subscription_Metadata")
if invalid_notifications_fk.empty:
    print(" All foreign keys in Notification_Messages are valid.\n")
else:
    print(" Invalid foreign keys in Notification_Messages:\n", invalid_notifications_fk)


 No primary key violation in Subscription_Info.

 No primary key violation in Payments_Info.

 No primary key violation in Subscription_Metadata.

 No primary key violation in Notification_Messages.

 FK Check: Payments_Info → Subscription_Info
 All foreign keys in Payments_Info are valid.

 FK Check: Notification_Messages → Subscription_Metadata
 All foreign keys in Notification_Messages are valid.



In [35]:
import pandas as pd
sub_metadata = pd.read_csv("Subscription_Metadata_CLEANED.csv")
sub_info = pd.read_csv("Subscription_Info_CLEANED.csv")

users = pd.read_csv("transformed_users.csv")
services = pd.read_csv("transformed_services.csv")
billing_cycles = pd.read_csv("transformed_billing_cycles.csv")
sub_status = pd.read_csv("transformed_subscription_status.csv")

def check_fk(child_df, child_col, parent_df, parent_col, label):
    invalid = child_df[~child_df[child_col].isin(parent_df[parent_col])]
    if invalid.empty:
        print(f"All {label} keys are valid.")
    else:
        print(f"Invalid {label} foreign keys:\n", invalid[[child_col]].drop_duplicates())

print(" Checking FKs for Subscription_Metadata...\n")
check_fk(sub_metadata, "user_id", users, "user_id", "sub_metadata → users")
check_fk(sub_metadata, "service_id", services, "service_id", "sub_metadata → services")
check_fk(sub_metadata, "billing_cycle_id", billing_cycles, "billing_cycle_id", "sub_metadata → billing_cycles")
check_fk(sub_metadata, "status_id", sub_status, "status_id", "sub_metadata → subscription_status")

print("\n Checking FKs for Subscription_Info...\n")
check_fk(sub_info, "user_id", users, "user_id", "sub_info → users")
check_fk(sub_info, "service_id", services, "service_id", "sub_info → services")
check_fk(sub_info, "billing_cycle_id", billing_cycles, "billing_cycle_id", "sub_info → billing_cycles")
check_fk(sub_info, "status_id", sub_status, "status_id", "sub_info → subscription_status")


 Checking FKs for Subscription_Metadata...

All sub_metadata → users keys are valid.
All sub_metadata → services keys are valid.
All sub_metadata → billing_cycles keys are valid.
All sub_metadata → subscription_status keys are valid.

 Checking FKs for Subscription_Info...

All sub_info → users keys are valid.
All sub_info → services keys are valid.
All sub_info → billing_cycles keys are valid.
All sub_info → subscription_status keys are valid.


In [44]:
random_seed = 42
users_df = pd.read_csv("transformed_users.csv")
categories_df = pd.read_csv("transformed_categories.csv")
services_df = pd.read_csv("transformed_services.csv")
billing_cycles_df = pd.read_csv("transformed_billing_cycles.csv")
subscription_status_df = pd.read_csv("transformed_subscription_status.csv")
subscriptions_df = pd.read_csv("transformed_subscriptions.csv")
payment_methods_df = pd.read_csv("transformed_payment_methods.csv")
payments_df = pd.read_csv("transformed_payments.csv")
notifications_df = pd.read_csv("transformed_notifications.csv")
users_sample = users_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
categories_sample = categories_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
services_sample = services_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
billing_cycles_sample = billing_cycles_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
subscription_status_sample = subscription_status_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
subscriptions_sample = subscriptions_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
payment_methods_sample = payment_methods_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
payments_sample = payments_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
notifications_sample = notifications_df.sample(frac=0.50, random_state=random_seed).reset_index(drop=True)
users_sample.to_csv("transformed_sampleusers.csv", index=False)
categories_sample.to_csv("transformed_samplecategories.csv", index=False)
services_sample.to_csv("transformed_sampleservices.csv", index=False)
billing_cycles_sample.to_csv("transformed_samplebilling_cycles.csv", index=False)
subscription_status_sample.to_csv("transformed_samplesubscription_status.csv", index=False)
subscriptions_sample.to_csv("transformed_samplesubscriptions.csv", index=False)
payment_methods_sample.to_csv("transformed_samplepayment_methods.csv", index=False)
payments_sample.to_csv("transformed_samplepayments.csv", index=False)
notifications_sample.to_csv("transformed_samplenotifications.csv", index=False)
print("50% Sample Data has been successfully created and saved for testing.")

50% Sample Data has been successfully created and saved for testing.
